In [20]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple

In [21]:
load_dotenv()

True

In [22]:
# ors_key = os.getenv("ORS_API_KEY")

# client = ors.Client(key= ors_key)

In [23]:
directory = os.getcwd()

os.chdir(directory)

hdbCentroids_df = pd.read_csv(r"..\data\hdb_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
mrt_stations_df = pd.read_csv(r"..\data\mrt_station_final.csv",usecols = [1,2,3])

In [24]:
hdbCentroids_df.info()
mrt_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   275 non-null    float64
 1   Longitude  275 non-null    float64
dtypes: float64(2)
memory usage: 4.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MRT.Name   175 non-null    object 
 1   Latitude   175 non-null    float64
 2   Longitude  175 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.2+ KB


In [25]:
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)

mrt_stations_df

,MRT.Name,Latitude,Longitude
12,ADMIRALTY MRT STATION,1.440589,103.800990
9,ALJUNIED MRT STATION,1.316433,103.882906
52,ANG MO KIO MRT STATION,1.369429,103.849455
115,BAKAU LRT STATION,1.387994,103.905415
54,BANGKIT LRT STATION,1.380022,103.772647
...,...,...,...
158,WOODLANDS SOUTH MRT STATION,1.427488,103.792730
80,WOODLEIGH MRT STATION,1.339190,103.870818
77,YEW TEE MRT STATION,1.397298,103.747358
79,YIO CHU KANG MRT STATION,1.381499,103.845171


In [26]:
#create a psuedo index for my residential centroids df
hdbCentroids_df['index'] = hdbCentroids_df.index

hdbCentroids_df['index']

#create a psuedo index for my residential centroids df
mrt_stations_df['index'] = mrt_stations_df.index

mrt_stations_df['index']

##create a dummy variable to cross join on 
hdbCentroids_df['join_key'] = "A"
mrt_stations_df['join_key'] = "A"

#Cross join to obtain combinations of all possible pairings between MRTs and Residential Centroids
combined_df = pd.merge(hdbCentroids_df, mrt_stations_df, on='join_key')

print(combined_df)

       Latitude_x  Longitude_x  index_x join_key                     MRT.Name  \
0        1.432477   103.791322        0        A        ADMIRALTY MRT STATION   
1        1.432477   103.791322        0        A         ALJUNIED MRT STATION   
2        1.432477   103.791322        0        A       ANG MO KIO MRT STATION   
3        1.432477   103.791322        0        A            BAKAU LRT STATION   
4        1.432477   103.791322        0        A          BANGKIT LRT STATION   
...           ...          ...      ...      ...                          ...   
48120    1.388649   103.901674      274        A  WOODLANDS SOUTH MRT STATION   
48121    1.388649   103.901674      274        A        WOODLEIGH MRT STATION   
48122    1.388649   103.901674      274        A          YEW TEE MRT STATION   
48123    1.388649   103.901674      274        A     YIO CHU KANG MRT STATION   
48124    1.388649   103.901674      274        A           YISHUN MRT STATION   

       Latitude_y  Longitud

In [27]:
#Defining a function that calculates the Euclidean Distance between two points using Haversine Method?
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c
    return distance

In [28]:
coordinate_pair = namedtuple('coordinate_pair',['lat_x','lon_x','lat_y','lon_y'])

In [29]:
#Apply function to dataframe and store distances in new column 'euclidean distance'
combined_df['euclidean_distance'] = haversine(combined_df['Latitude_x'], combined_df['Longitude_x'], combined_df['Latitude_y'], combined_df['Longitude_y'])

#Group by residential centroid, filter out the closest MRT by distance for each centroid into another dataframe and reset its index
result_df = combined_df.loc[combined_df.groupby('index_x')['euclidean_distance'].idxmin()].reset_index()

result_df

#Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
# result_df['coordinate_pair'] = list(zip(result_df['Longitude_x'], result_df['Latitude_x'], result_df['Longitude_y'], result_df['Latitude_y']))
# result_df['coordinate_pair'] = result_df['coordinate_pair'].apply(lambda x: [[x[0], x[1]], [x[2], x[3]]])
result_df['coordinate_pair'] = result_df.apply(lambda x: coordinate_pair(x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']), axis=1)

#create an empty column 'route' to later store query response
result_df['route'] = np.nan

In [30]:
one_map_email = os.getenv("ONE_MAP_EMAIL")
one_map_password = os.getenv("ONE_MAP_PASSWORD")
payload = {
        "email": one_map_email,
        "password": one_map_password
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

In [31]:
onemap = OneMap(api_key)

In [32]:
def get_route(coordinate_pair):
    time.sleep(1)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="cycle")
    except Exception as e:
        print(f"Error: {e}")
        return None
    
    # def get_route(coordinate_pair):
#     time.sleep(2)
#     try:
#         return client.directions(coordinate_pair, profile='cycling-regular', format='geojson', validate=False)
#     except Exception as e:
#         print(f"Error: {e}")
#         return None

In [34]:
result_df['route'] = result_df['coordinate_pair'].apply(get_route)

In [35]:
def get_distance(route):
    try:
        return route['route_summary']['total_distance']/1000 #convert m to km
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['distance'] = result_df['route'].apply(get_distance)

def get_time(route):
    try:
        return route['route_summary']['total_time']/60 #convert second to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None
    

result_df['duration'] = result_df['route'].apply(get_time)

# def get_distance(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['distance']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None

# result_df['distance'] = result_df['route'].apply(get_distance)

# def get_time(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['duration']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None
    

# result_df['duration'] = result_df['route'].apply(get_time)

In [36]:
def get_centroid_name(row):
    geocode = onemap.reverseGeocode.revGeoCode(row['Latitude_x'], row['Longitude_x'])
    if geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "NIL":
        return geocode['GeocodeInfo'][0]['BUILDINGNAME']
    else:
        if geocode['GeocodeInfo'][0]['BLOCK'] != "NIL":
            return geocode['GeocodeInfo'][0]['BLOCK'] + " " + geocode['GeocodeInfo'][0]['ROAD']
        else:
            return geocode['GeocodeInfo'][0]['ROAD']

In [37]:
result_df['centroid_name'] = result_df.apply(get_centroid_name, axis=1)

In [38]:
new_result_df = result_df. loc[:, result_df. columns != 'route']
new_result_df.to_csv(r"..\data\HDB_Centroid_MRT pairing data.csv")
routes = result_df['route'].copy(deep = True)
routes.to_json(r'..\data\HDB_MRT_routes.json', orient='records')

In [39]:
result_df[result_df['route'].isna()]

,index,Latitude_x,Longitude_x,index_x,join_key,MRT.Name,Latitude_y,Longitude_y,index_y,euclidean_distance,coordinate_pair,route,distance,duration,centroid_name


In [40]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               275 non-null    int64  
 1   Latitude_x          275 non-null    float64
 2   Longitude_x         275 non-null    float64
 3   index_x             275 non-null    int64  
 4   join_key            275 non-null    object 
 5   MRT.Name            275 non-null    object 
 6   Latitude_y          275 non-null    float64
 7   Longitude_y         275 non-null    float64
 8   index_y             275 non-null    int64  
 9   euclidean_distance  275 non-null    float64
 10  coordinate_pair     275 non-null    object 
 11  route               275 non-null    object 
 12  distance            275 non-null    float64
 13  duration            275 non-null    float64
 14  centroid_name       275 non-null    object 
dtypes: float64(7), int64(3), object(5)
memory usage: 32.4+ KB